<a href="https://colab.research.google.com/github/zaraprvz/Diabetes-Prediction-Capstone-Project/blob/main/Online_Payments_Fraud_Detection_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [11]:
!kaggle datasets download -d chitwanmanchanda/fraudulent-transactions-data

Dataset URL: https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data
License(s): CC0-1.0
 98% 174M/178M [00:02<00:00, 95.2MB/s]
100% 178M/178M [00:02<00:00, 70.6MB/s]


In [12]:
#unzip file
!unzip /content/fraudulent-transactions-data.zip

Archive:  /content/fraudulent-transactions-data.zip
  inflating: Fraud.csv               


In [14]:
data = pd.read_csv('/content/Fraud.csv')
print(data.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [16]:
#find missing values
print(data.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [17]:
#Exploring the transaction type
print(data.type.value_counts())

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64


In [19]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

#plot a pie
import plotly.express as px
figure = px.pie(data,
                values = quantity,
                names = transactions,
                hole = 0.5,
                title = "Distribution of Transaction Type")
figure.show()


In [22]:
#Lets check correlation
correlation = data.corr(numeric_only=True)
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


Now let’s transform the categorical features into numerical. Here I will also transform the values of the isFraud column into No Fraud and Fraud labels to have a better understanding of the output:

In [65]:
data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})
data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
print(data.head())

Empty DataFrame
Columns: [step, type, amount, nameOrig, oldbalanceOrg, newbalanceOrig, nameDest, oldbalanceDest, newbalanceDest, isFraud, isFlaggedFraud, isfraud]
Index: []


Online Payments Fraud Detection Model

In [66]:
#Instead of dropping all rows with missing values, try filling them
data = data.fillna(method='ffill')  # Forward fill missing values

from sklearn.model_selection import train_test_split

# Features
x = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])

# Target
y = np.array(data[["isFraud"]])



In [67]:
# Now proceed with train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)#Now let’s train the online payments fraud detection model:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree model
model = DecisionTreeClassifier()

# Train the model on the training data
model.fit(xtrain, ytrain)
# Evaluate the model's accuracy on the test data
print(model.score(xtest, ytest))

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [43]:
#Now let’s classify whether a transaction is a fraud or not by feeding about a transaction into the model:
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(model.predict(features))

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.